In [ ]:
import os

from importlib.metadata import version

try:
    import xradio

    print('XRADIO version', version('xradio'), 'already installed.')
except ImportError as e:
    print(e)
    print('Installing XRADIO')

    os.system("pip install xradio")

    import xradio
    print('xradio version', version('xradio'), ' installed.')

In [ ]:
from xradio.vis._vis_utils._ms._tables.read import (
    read_generic_table,
    make_freq_attrs,
    convert_casacore_time,
    extract_table_attributes,
)

#infile = 'Antennae_North.cal.lsrk.ms/SPECTRAL_WINDOW'
#infile = 'Antennae_North.cal.lsrk.ms/FIELD'
#infile = 'Antennae_North.cal.lsrk.ms/ANTENNA'
infile = 'Antennae_North.cal.lsrk.ms'
main_table_attrs = extract_table_attributes(infile)
#print(main_table_attrs["column_descriptions"].keys())

casacore_to_xradio_measure_type = {'quanta':{'type':'quanta','Ref':None},
                                   'direction':{'type':'sky_coord','Ref':'frame'},
                                   'epoch':{'type':'time','Ref':'scale'},
                                   'frequency':{'type':'spectral_coord','Ref':'frame'},
                                   'position':{'type':'earth_location','Ref':'ellipsoid'},
                                   'uvw':{'type':'uvw','Ref':'frame'}}

casacore_to_xradio_ref = {'J2000':'FK5', 'ITRF':'GRS80'}

def column_description_casacore_to_xradio_measure(casacore_column_description,time_format='unix'):
    xradio_measure={}
    if 'MEASINFO' in casacore_column_description['keywords']:
        xradio_measure['type'] = casacore_to_xradio_measure_type[casacore_column_description['keywords']['MEASINFO']['type']]['type']
        xradio_measure['units'] = casacore_column_description['keywords']['QuantumUnits']

        casa_ref = casacore_column_description['keywords']['MEASINFO']['Ref']

        if casa_ref in casacore_to_xradio_ref:
            casa_ref = casacore_to_xradio_ref[casa_ref]
        xradio_measure[casacore_to_xradio_measure_type[casacore_column_description['keywords']['MEASINFO']['type']]['Ref']] = casa_ref       #print(casacore_column_description['keywords']['MEASINFO'])
        #print(casacore_column_description['keywords'])

        if xradio_measure['type'] == 'time':
            xradio_measure['format'] = 'unix'

    
        
    

    #print(xradio_measure)
    
    return xradio_measure
    

#print(main_table_attrs["column_descriptions"]['TIME'])
#clean and the im tool ignore the reference frame claimed by the UVW column (it is often mislabelled as ITRF when it is really FK5 (J2000)) and instead assume the (u, v, w)s are in the same frame as the phase tracking center. calcuvw does not yet force the UVW column and field centers to use the same reference frame! Blank = use the phase tracking frame of vis.

for name,description in main_table_attrs["column_descriptions"].items():
    xradio_measure = column_description_casacore_to_xradio_measure(casacore_column_description=description)
    print(name,xradio_measure)
    
#column_description_casacore_to_xradio_measure(casacore_column_description=main_table_attrs["column_descriptions"]['UVW'])

# Download ms

In [ ]:
from xradio.data.datasets import download
download(file='Antennae_North.cal.lsrk.ms',source='dropbox')

# Convert ms

In [1]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

partition_scheme="ddi_intent_field"
#partition_scheme="ddi_state"

chunks_on_disk = {'frequency':45}
infile = "Antennae_North.cal.lsrk.ms"
outfile = "Antennae_North.cal.lsrk.vis.zarr"
convert_msv2_to_processing_set(infile=infile, outfile=outfile, 
                               partition_scheme=partition_scheme,
                               parallel=False,overwrite=True,
                               chunks_on_disk=chunks_on_disk)


# Lazy Read PS

In [ ]:
ps_name = '/Users/jsteeb/Dropbox/Data/Antennae_North.cal.lsrk.vis.zarr'

from xradio.vis.read_processing_set import read_processing_set
#sel_parms = {'xds_id':[0,1,2]}
intents = ["OBSERVE_TARGET#ON_SOURCE"]
ps = read_processing_set("Antennae_North.cal.lsrk.vis.zarr",
                        intents=intents)
ps.summary()

# A single ms v4

In [ ]:
ps['Antennae_North.cal.lsrk_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1']

# Attributes of Data Arrays and Coordinates

All **data arrays** and **coordinates** can have measures information in their attributes section along with other relevant metadata. 

List of measures:
https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1504318014

Measures dictionary format (based on https://docs.xarray.dev/en/stable/generated/xarray.DataArray.from_dict.html):
{"dims":..., "data":..., "attrs":measures_dict} 


## Frequency Coordinate Example

The ```frequency``` coordinate is a spectral coord measure (keys: type,units,reference_frame) but also contains the channel_width, effective_channel_width, spectral_window_name, and reference_frequency. Any metadata that is a non-id number should be in the relevant measures dictionary (for example channel_width and reference_frequency.) 

In [ ]:
ps['Antennae_North.cal.lsrk_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1'].frequency

In [ ]:
import json
print(json.dumps(ps['Antennae_North.cal.lsrk_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1'].frequency.attrs,indent=4))

## Time Example
Another example is the time coordinate that is a time measure (keys: type, units, time_scale, format) but also contains integration_time and effective_integration_time.

In [ ]:
import json
print(json.dumps(ps['Antennae_North.cal.lsrk_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1'].time.attrs,indent=4))

# Info Attributes

The ms v4 also allows for info dictionaries in the attribute section of the dataset (used when data is not n-dimensional). Just like the non-id number metadata in coordinates and data arrays the relevant measure should be used. An example is the field_info where the delay_direction, phase_direction, and reference_direction are stored as sky_coord measures (keys: type, units, reference_frame).

In [ ]:
import json
print(json.dumps(ps['Antennae_North.cal.lsrk_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1'].field_info, indent=4))

In [ ]:
ps['Antennae_North.cal.lsrk_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1'].attrs['antenna_xds']